In [2]:
# import textacy
import pandas as pd
import pysrt
from datetime import datetime
import re

In [3]:
import spacy
import textacy
de = textacy.load_spacy_lang("de_core_news_sm", disable=("parser",))
nlp = spacy.load("de_core_news_sm")

In [4]:
from os import listdir
from os.path import isfile, join, isdir
subfolders = [f for f in listdir("./Subtitles") if isdir(join("./Subtitles", f))]
subfolders[1] #check

'Biohackers__show'

### Main script

In [11]:
for sf in subfolders: #alternatively select the new show that has been added if you don't want to overwrite existing shows' files
    show_name = (sf.split('__')[0]).replace('_', ' ')
    show_dir = f"./Subtitles/{sf}"
    show_files = [fl for fl in listdir(show_dir) if isfile(join(show_dir, fl))]
    for srt in show_files:
        season_name = show_name + ' ' + srt.split('__')[1][0:3]
        episode_name = show_name + ' ' + srt.split('__')[1][3:6]
        srt_dir = f"{show_dir}/{srt}"
        subs = pysrt.open(srt_dir)

        episode_subs_list = []
        lines_to_txt = []
        for sub in subs:
            
            line = sub.text_without_tags 
            #ensure the subtitle is actual dialogue:
            if (not ((line[0] == '[' or line[0] == '♪' or line[0:2] == '-[') and (line[-1] == ']' or line[-1] == '♪'))) and ('netflix' not in line.lower()):

                #index:
                line_index = sub.index

                #parse timestamp:
                sub_timestamp_start = f"{sub.start.hours}:{sub.start.minutes}:{sub.start.seconds},{sub.start.milliseconds}"
                sub_timestamp_end = f"{sub.end.hours}:{sub.end.minutes}:{sub.end.seconds},{sub.end.milliseconds}"

                #parse dialogue:
                line = line.replace('\n', ' ') #removes new line indicators
                line = line.replace('♪', '') #removes the music symbol
                line = line.replace('-', '') #removes the dash symbol
                line = line.replace(':', ',') #removes the colon symbol
                line = re.sub(r'\[.*?\]', '', line) #removes brackets and any string inside them
                line = line.strip() #removes leading and trailing whitespace
                line_lower = line.lower()

                doc_line = textacy.make_spacy_doc(line, lang=de)
                doc_line_list = ngrams = list(textacy.extract.basics.ngrams(doc_line, 1, min_freq=1, filter_stops = False, filter_punct = True))
                doc_line_list = [str(i) for i in doc_line_list]
                doc_line_list2 = [i.lower() for i in doc_line_list]
                doc_line_list_set = list(set(doc_line_list2))

                doc_line_v2 = nlp(line)
                doc_line_v2_text = []
                doc_line_v2_lemmas = []
                doc_line_v2_POS_tags = []
                
                for toke in doc_line_v2:
                    doc_line_v2_text.append(toke.text)
                    doc_line_v2_lemmas.append(toke.lemma_)
                    doc_line_v2_POS_tags.append(toke.pos_)

                doc_line_v2_lemmas_pattern = ' '.join(doc_line_v2_lemmas)
                doc_line_v2_POS_tags_pattern = ' '.join(doc_line_v2_POS_tags) 

                # words_in_line = re.findall(r'\w+', line.lower()) #adds each word from the dialogue to a list
                # words_in_line = list(set(words_in_line)) #removes duplicates from that same list
                

                #establish dictionary:
                sub_dict = {'Unique_ID': srt, 'Show_Name': show_name, 'Season': season_name, 'Episode': episode_name, 
                'Line_Index': line_index, 'Start': sub_timestamp_start, 'End': sub_timestamp_end,
                'Dialogue': line, 'Dialogue_lower': line_lower, 'Dialogue_Set_Words': doc_line_list_set, 'Dialogue_Words': doc_line_list, 
                'Dialogue_Tokens_Text': doc_line_v2_text, 'Dialogue_Tokens_Lemmas': doc_line_v2_lemmas, 'Dialogue_Tokens_POS_Tags': doc_line_v2_POS_tags,
                'Dialogue_Lemmas_Pattern': doc_line_v2_lemmas_pattern, 'Dialogue_POS_Tags_Pattern': doc_line_v2_POS_tags_pattern}
                episode_subs_list.append(sub_dict)

                #adding each dialogue line to a list, to later be concatenated as a txt file for doc parsing using spacy and textacy
                lines_to_txt.append(line)

        df = pd.DataFrame(episode_subs_list)
        csv_file_name = srt.replace(".srt", ".csv")
        csv_write_path = f"./German/CSV/{sf}/{csv_file_name}"
        df.to_csv(csv_write_path, index=False)

        #save as episode dialogues as txt file:
        txt_file_dialogue = ' '.join(lines_to_txt)
        txt_file_name = srt.replace(".srt", ".txt")
        txt_write_path = f"./German/Docs/{sf}/{txt_file_name}"
        file1 = open(txt_write_path,"w")#write mode
        file1.write(txt_file_dialogue)
        file1.close()
